In [ ]:
import math
import sys
import os

import IPython
import matplotlib.pylab as plt
import numpy as np
from scipy import signal

%load_ext autoreload
%autoreload 2

In [ ]:
from crazyflie_description_py.parameters import TUKEY_ALPHA, N_BUFFER

tukey_window = signal.tukey(N_BUFFER, alpha=TUKEY_ALPHA)
hann_window = signal.hann(N_BUFFER)
flattop_window = signal.flattop(N_BUFFER, True)

In [ ]:
plt.plot(hann_window, label="hann")
plt.plot(tukey_window, label="tukey")
plt.plot(flattop_window, label="flattop")
plt.legend()

In [ ]:
FS = 32000
f = 1000
noise = 1.0
s = np.sin(2*np.pi*f*np.arange(buffer_length)/FS) + np.random.normal(scale=noise, size=buffer_length)

freq = np.fft.rfftfreq(len(ramp_window), 1/FS)
plt.semilogy(freq, np.abs(np.fft.rfft(s*hann_window))/buffer_length, label='hann')
plt.semilogy(freq, np.abs(np.fft.rfft(s*tukey_window))/buffer_length, label='tukey')
plt.semilogy(freq, np.abs(np.fft.rfft(s*flattop_window))/buffer_length, label='flattop')
plt.xlim(f-500, f+500)

In [ ]:
out_dir = "../firmware/audio_shield_firmware/Core/Inc"

save_dict = {
    "hann_window": hann_window,
    "tukey_window": tukey_window,
    "flattop_window": flattop_window,
}

max_int16 = 2**(16-1) - 1
print(max_int16)

for key, arr in save_dict.items():
    
    print(np.min(arr), np.max(arr))
    arr *= max_int16 
    arr = np.round(arr)
    print(arr)
    
    fname = f"{out_dir}/{key}.h"
    with open(fname, "w+") as f:
        f.write(f"#ifndef __{key.upper()}_H \n#define __{key.upper()}_H\n\n")

        f.write(f"int16_t {key}[{buffer_length}] = " + r"{")
        [f.write(f"{s:.0f}, ",) for s in arr[:-1]]
        [f.write(f"{s:.0f}",) for s in arr[-1:]]
        f.write(r"};" + " \n\n")

        f.write(f"#endif /* __{key.upper()}_H */")
    print('wrote to', fname)